<a href="https://colab.research.google.com/github/harirm85/Hari_ML_workshop/blob/main/Home_Credit_Default_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount drive and connect to Kaggle

In [2]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Copy the Jason file and set security
!mkdir -p ~/.kaggle
!cp '/content/drive/MyDrive/Kaggle/kaggle.json' ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# # Upload Jason file from system manually 
# from google.colab import files
# files.upload()
# # Copy the Jason file and set security
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [10]:
#Download data
!kaggle competitions download -c home-credit-default-risk

100% 688M/688M [00:05<00:00, 200MB/s]
100% 688M/688M [00:05<00:00, 132MB/s]


In [11]:
import zipfile

# Specify the path to the downloaded zip file
zip_path = "/content/home-credit-default-risk.zip"

# Specify the directory to extract the files to
extract_path = "/content/home-credit-default-risk"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
# import shutil

# # Specify the path to the directory where you want to save the dataset
# save_path = "/content/drive/MyDrive/Kaggle/home-credit-default-risk"

# # Create a copy of the extracted dataset in the save path
# shutil.copytree(extract_path, save_path)

# Verify data

In [13]:
import pandas as pd

# Load the dataset
bureau_df = pd.read_csv('/content/home-credit-default-risk/bureau.csv')

bureau_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN
